In [139]:
!pip install pandas sklearn gym

ERROR: Could not find a version that satisfies the requirement Itertools (from versions: none)
ERROR: No matching distribution found for Itertools


In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from gym import spaces, Env
import numpy as np

In [3]:
class LineAgent:
    def __init__(self):
        self.vectors = np.zeros(shape=(2, 10))

    def predict(self, actions):
        self.vectors = self.vectors + actions


In [161]:
import math
from operator import truediv

from numpy import NaN, nan


class VectorEnv(Env):
    def __init__(self, episodes):
        super(VectorEnv, self).__init__()
        self.action_space = spaces.Box(low=-5, high=5, shape=(10, 2))
        self.observation_space = (38, 4)
        self.episodes = episodes
        self.reset()

    def step(self, action):
        self.t += 1
        done = self.t > 100
        self.state = self.state + action

        # Calculate reward based on state and episode
        reward = 0
        state_with_start = np.concatenate(([self.start], self.state[1:]))
        state_with_end = np.concatenate((self.state[:-1], [self.end]))
        for (v1, v2) in zip(state_with_start, state_with_end):
            for box in self.episode:
                reward -= -1000 if self.intersects(v1, v2, box) else 0

        # Calculate distance of linesegments and reward for shorter lines

        return self.episode, reward, done, {}

    def reset(self):
        self.state = np.zeros(shape=(10, 2))
        episode = self.episodes.sample()
        self.episode = self.chunks(episode.values.flatten(), 4)
        self.t = 0
        self.start = self.episode[0][:2]
        self.end = self.episode[-1][:2]
        return self.state

    def intersects(self, v1, v2, box):
        x1 = v1[0]
        y1 = v1[1]
        x2 = v2[0]
        y2 = v2[1]
        bx = box[0]
        by = box[1]
        bw = box[2]
        bh = box[3]
        box_lines = [ (bx, by, bx, by + bh), (bx, by - bh, bx + bw, by - bh), (bx + bw, by - bh, bx - bw, by), (bx, by, bx + bh, by) ]
        result = False
        for (bx_, by_, bx2_, by2_) in box_lines:
            if result:
                return result
            else:
                result = self.ccw(x1, y1, bx_, by_, bx2_, by2_) != self.ccw(x2, y2, bx_, by_, bx2_, by2_) and self.ccw(x1, y1, x2, y2, bx_, by_) != self.ccw(x1, y1, x2, y2, bx2_, by2_)
        
        return result

    def chunks(self, lst, n):
        result = []
        for i in range(0, len(lst), n):
            if math.isnan(lst[i]):
                break
            else:
                result.append(lst[i:i + n])
        return result

    def ccw(self, x1, y1, x2, y2, x3, y3):
        return (y3-y1) * (x2-x1) > (y2-y1) * (x3-x1)

In [162]:
dataset = pd.read_csv(os.path.join(os.getcwd(), "../../data/vectors/vector.csv"), header = None, delimiter=',', names = list(range(150)))

In [163]:
env = VectorEnv(dataset)

obj, r, d, x = env.step(np.arange(0, 20, dtype=int).reshape(10, 2))